## Naver Movie
- robots.txt 설정으로 크롤링 안됨.

### 프로젝트 생성

In [2]:
!scrapy startproject naver_movie

Error: scrapy.cfg already exists in C:\jupyter_project\data_science\practice_web\naver_movie


:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'pyasn1_modules''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


### items 설정
- https://movie.naver.com/movie/running/current.nhn
- 제목, 관객수, 평점

In [3]:
!cat naver_movie/naver_movie/items.py

import scrapy


class NaverMovieItem(scrapy.Item):
    title = scrapy.Field()
    count =  scrapy.Field()
    star = scrapy.Field()


In [4]:
%%writefile naver_movie/naver_movie/items.py
import scrapy


class NaverMovieItem(scrapy.Item):
    title = scrapy.Field()
    count =  scrapy.Field()
    star = scrapy.Field()

Overwriting naver_movie/naver_movie/items.py


### xpath 확인

In [5]:
import requests
import scrapy
from scrapy.http import TextResponse

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'pyasn1_modules''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


In [61]:
req=  requests.get('https://movie.naver.com/movie/running/current.nhn')
response = TextResponse(req.url, body=req.text, encoding='utf-8')

In [62]:
links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()
len(links), links[0]

(109, '/movie/bi/mi/basic.nhn?code=187310')

In [8]:
link = response.urljoin(links[0])
link

'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187310'

In [9]:
# 상세 페이지 데이터 수집
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding='utf-8')

In [10]:
title = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
count = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0]
star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()

star = ''.join(star)
title, count, star

('미나리', '346,304명', '8.41')

### spider.py 작성

In [76]:
%%writefile naver_movie/naver_movie/spiders/spider.py

import scrapy
from naver_movie.items import NaverMovieItem

class Spider(scrapy.Spider):
    name = 'NaverMovie'
    allow_domain = ['movie.naver.com/']
    start_urls = ['https://movie.naver.com/movie/running/current.nhn']
    
    def parse(self, response):
        links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()
        for link in links:
            link = response.urljoin(link)
        #link = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()[0]
        #link = response.urljoin(link)
            yield scrapy.Request(link, callback = self.parse_page_contents)
            
    def parse_page_contents(self, response):
        item = NaverMovieItem()
        item['title'] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
        try:
            item['count'] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0]
        except:
            item['count'] = '0명'
        star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
        item['star'] = ''.join(star)
        yield item

Overwriting naver_movie/naver_movie/spiders/spider.py


### scrapy 실행

In [77]:
%%writefile run.sh
cd naver_movie
scrapy crawl NaverMovie -o naver_movie.csv

Overwriting run.sh


In [73]:
!chmod +x run.sh

In [49]:
!run.sh

### settings.py 파일 변경
- robots.txt에 의해 forbidden됨

In [65]:
!head -n 21 naver_movie/naver_movie/settings.py | tail -n 5

#USER_AGENT = 'naver_movie (+http://www.yourdomain.com)'

# Obey robots.txt rules
ROBOTSTXT_OBEY = False



In [66]:
!sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' naver_movie/naver_movie/settings.py

In [78]:
!run.sh

In [82]:
movie_data = pd.read_csv('./naver_movie/naver_movie.csv')
movie_data

,count,star,title
0,0명,6.50,리틀드래곤 코코넛2: 정글대탐험
1,0명,9.40,해리 포터와 불의 잔
2,0명,10.00,UFO 스케치
3,0명,7.83,프라미싱 영 우먼
4,0명,8.77,분노
...,...,...,...
104,0명,7.86,더 레이서
105,"141,745명",7.34,카오스 워킹
106,0명,NaN,암모나이트
107,"42,814명",9.36,중경삼림


In [85]:
'미나리' in movie_data.title.values

True